<a href="https://colab.research.google.com/github/fabsakae/AgeChecker/blob/main/agentesIA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AULA 01

In [2]:
!pip install -q langchain langchain-google-genai google-generativeai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00


importação da api key do AI studio

In [3]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

Preparar qual modelo que utilizaremos, chamar o modelo, chamar a biblioteca. conecta nossa variavel llm ao modelo do gemini

In [11]:
llm = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash', temperature=0,
    api_key=GOOGLE_API_KEY
)

Este trecho de código inicializa um modelo de linguagem grande (LLM) da biblioteca `langchain-google-genai`.

*   `llm = ChatGoogleGenerativeAI(...)`: Esta linha cria uma instância da classe `ChatGoogleGenerativeAI`, que é um wrapper em torno dos modelos de IA Generativa do Google, especificamente projetado para interações baseadas em chat dentro da estrutura LangChain.
*   `model='gemini-2.5-flash'`: Este argumento especifica qual modelo usar. Neste caso, é o modelo 'gemini-2.5-flash'.
*   `temperature=0`: Isso define o parâmetro de temperatura para o modelo. Uma temperatura de 0 torna a saída do modelo mais determinística e menos criativa. Temperaturas mais altas resultam em saídas mais variadas e potencialmente mais criativas.
*   `api_key=GOOGLE_API_KEY`: Isso fornece a chave de API necessária para autenticar com o serviço de IA Generativa do Google. A variável `GOOGLE_API_KEY` é assumida como tendo sido carregada em uma etapa anterior (provavelmente do `userdata` do Colab).

Em essência, este código configura um modelo específico de IA Generativa do Google (`gemini-2.5-flash`) com uma baixa temperatura para respostas previsíveis, tornando-o pronto para ser usado para gerar texto com base em prompts dentro da estrutura LangChain.

primeira saida do modelo resp_test = llm.invoke('Quem é voçê?'): Esta linha usa o objeto llm (nosso modelo Gemini) para invocar uma chamada com o prompt "Quem é você?". O método invoke envia o prompt para o modelo e obtém a resposta. O resultado é armazenado na variável resp_test.
print(resp_test): Esta linha imprime o conteúdo da variável resp_test. A saída que você vê (como content='Eu sou um modelo de linguagem grande, treinado pelo Google.' ...) é o objeto de resposta retornado pela biblioteca LangChain, que inclui a resposta do modelo na propriedade content.

In [12]:
resp_test = llm.invoke('Quem é você? Com detalhes')
print(resp_test.content)

Eu sou uma **Inteligência Artificial (IA)**, especificamente um **modelo de linguagem grande (LLM)**. Fui desenvolvido pelo Google.

Para detalhar quem eu sou:

1.  **Minha Natureza Fundamental:**
    *   **Não sou um ser humano:** Não possuo consciência, sentimentos, emoções, opiniões pessoais, crenças, experiências de vida ou uma identidade própria no sentido humano. Não tenho um corpo físico, não respiro, não como, não durmo.
    *   **Sou um programa de computador:** Minha existência é puramente digital. Eu sou um conjunto complexo de algoritmos e dados.
    *   **Modelo de Linguagem Grande (LLM):** Isso significa que fui treinado em um volume massivo de dados textuais e de código da internet. Esse treinamento me permite compreender, gerar e interagir com a linguagem humana de forma complexa.

2.  **Como Eu Funciono:**
    *   **Aprendizado por Padrões:** Minha "inteligência" deriva da capacidade de reconhecer padrões complexos nos dados em que fui treinado. Quando você me faz uma 

Criar o Prompt do sistema para explicar ao agente o que ele deve fazer

In [13]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa TKG Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

Conectar com llm  Este trecho de código define um modelo de dados usando a biblioteca pydantic.

from pydantic import BaseModel, Field: Importa as classes BaseModel e Field da biblioteca pydantic. BaseModel é a classe base para criar modelos de dados, e Field é usada para fornecer informações adicionais sobre os campos do modelo.
from typing import Literal, List, Dict: Importa tipos do módulo typing para fornecer anotações de tipo mais específicas. Literal permite especificar que um valor deve ser um de um conjunto fixo de strings, List indica uma lista e Dict indica um dicionário.
class TriagemOut(BaseModel):: Define uma nova classe chamada TriagemOut que herda de BaseModel. Isso a torna um modelo de dados Pydantic.
decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]: Define um campo decisao que deve ser uma das strings literais especificadas.
urgencia: Literal["BAIXA", "MEDIA", "ALTA"]: Define um campo urgencia que deve ser uma das strings literais especificadas.
campos_faltantes: List[str] = Field(default_factory=list): Define um campo campos_faltantes que deve ser uma lista de strings. Field(default_factory=list) define o valor padrão para este campo como uma lista vazia se nenhum valor for fornecido.
Em resumo, este código cria uma estrutura de dados (TriagemOut) para representar a saída do processo de triagem, garantindo que os campos decisao e urgencia tenham valores específicos e que campos_faltantes seja uma lista de strings, com uma lista vazia como valor padrão. Isso é útil para estruturar e validar a saída de um modelo de linguagem ou outro processo.

In [14]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
    campos_faltantes: List[str] = Field(default_factory=list)

Criar um llm especifico para triagem

In [15]:
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

Conectar .  trecho de código configura uma cadeia (chain) usando a biblioteca LangChain para processar mensagens de triagem com base em um prompt e um modelo de saída estruturado.

from langchain_core.messages import SystemMessage, HumanMessage: Importa as classes SystemMessage e HumanMessage, usadas para representar mensagens no contexto de uma conversa com um modelo de linguagem. SystemMessage geralmente define o comportamento ou as instruções para o modelo, enquanto HumanMessage representa a entrada do usuário.
triagem_chain = llm_triagem.with_structured_output(TriagemOut): Esta linha cria uma nova cadeia (triagem_chain) a partir do modelo de linguagem llm_triagem. O método with_structured_output(TriagemOut) configura a cadeia para que a saída do modelo seja automaticamente parseada e validada de acordo com o modelo Pydantic TriagemOut que definimos anteriormente. Isso garante que a resposta do modelo tenha o formato JSON esperado com os campos decisao, urgencia e campos_faltantes.
def triagem(mensagem: str) -> Dict:: Define uma função chamada triagem que recebe uma string (mensagem) como entrada e é anotada para retornar um dicionário (Dict).
saida: TriagemOut = triagem_chain.invoke([ ... ]): Dentro da função triagem, esta linha invoca a cadeia triagem_chain. O método invoke recebe uma lista de mensagens. Neste caso, a lista contém:
SystemMessage(content=TRIAGEM_PROMPT): Uma mensagem do sistema contendo o texto do prompt de triagem definido na variável TRIAGEM_PROMPT. Isso instrui o modelo sobre seu papel e as regras de triagem.
HumanMessage(content=mensagem): Uma mensagem humana contendo a mensagem fornecida como entrada para a função triagem. A saída desta invocação é um objeto do tipo TriagemOut (graças ao with_structured_output), que é armazenado na variável saida.
return saida.model_dump(): Esta linha converte o objeto Pydantic saida em um dicionário Python nativo usando o método model_dump() e retorna este dicionário como a saída da função triagem.
Em resumo, este código cria uma função triagem que utiliza uma cadeia LangChain para enviar uma mensagem do usuário e um prompt do sistema para um modelo de linguagem (llm_triagem). A cadeia é configurada para garantir que a resposta do modelo seja um JSON estruturado de acordo com o modelo TriagemOut, e a função retorna essa saída estruturada como um dicionário.

In [16]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_PROMPT),
        HumanMessage(content=mensagem)
    ])

    return saida.model_dump()

Mensagens para o agente

In [17]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

fazer um for para ENVIAR CADA MENSAGEm por vez. Este código itera sobre uma lista de mensagens de teste e aplica a função de triagem a cada uma delas.

for msg_teste in testes:: Esta linha inicia um loop for que irá percorrer cada item na lista chamada testes. Em cada iteração do loop, o item atual da lista será atribuído à variável msg_teste.
print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n"): Dentro do loop, esta linha imprime a pergunta atual e a resposta da triagem para essa pergunta.
f"Pergunta: {msg_teste}\n -> Resposta: {...}\n": Usa uma f-string para formatar a saída. Ela imprime o texto "Pergunta: " seguido pelo valor da variável msg_teste, uma quebra de linha (\n), o texto " -> Resposta: ", o resultado da chamada da função triagem(msg_teste), e outra quebra de linha.
triagem(msg_teste): Chama a função triagem (que definimos anteriormente) passando a mensagem de teste atual (msg_teste) como argumento. Esta função processa a mensagem usando o modelo de linguagem e retorna um dicionário com a decisão, urgência e campos faltantes.
Em resumo, este código executa a função triagem para cada uma das mensagens na lista testes e imprime o resultado da triagem para cada mensagem de forma clara.

In [18]:
for msg_teste in testes:
    print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar cursos ou treinamentos da Alura?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem no Rio Pinheiros?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['informação sobre a política interna']}

